# DECISION TREE Y RANDOM FOREST
---

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import pickle

import warnings
warnings.filterwarnings('once')

/home/sanfermen/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_est = pd.read_csv("../datos/08-bikes_enc_est_sinoutliers.csv", index_col = 0)
df_est.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,1,0,0,0,1,2,-0.691698,1.254842,-0.360109,654
1,0,1,0,7,2,2,2,-0.752316,0.465708,0.867064,670
2,0,1,1,7,2,2,4,-1.759976,-1.395307,0.863860,1229
3,1,1,1,7,1,1,4,-1.620681,-0.293988,-0.362199,1454
4,1,1,2,7,1,2,4,-1.515533,-1.397579,0.008406,1518


In [3]:
df_est.isnull().sum()

season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
atemp         0
hum           0
windspeed     0
registered    0
dtype: int64

In [4]:
# Creamos una clase para poder hacer el decision tree y el random forest más rápido
class Regres_lineal:
    def __init__(self, dataf, col_pred):
        
        self.dataf = dataf
        self.col_pred = col_pred

    def tree_param(self):
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

        arbol = DecisionTreeRegressor(random_state =0)
    
        arbol.fit(x_train, y_train)

        max_feat = np.sqrt(len(x_train.columns))
        max_dep = arbol.tree_.max_depth
        return(max_feat, max_dep)

    def metricas(self, y_te, y_tr, y_te_pred, y_tr_pred, tipo_modelo):
        resultados = {'MAE': [mean_absolute_error(y_te, y_te_pred), mean_absolute_error(y_tr, y_tr_pred)],
                    'MSE': [mean_squared_error(y_te, y_te_pred), mean_squared_error(y_tr, y_tr_pred)],
                    'RMSE': [np.sqrt(mean_squared_error(y_te, y_te_pred)), np.sqrt(mean_squared_error(y_tr, y_tr_pred))],
                    'R2':  [r2_score(y_te, y_te_pred), r2_score(y_tr, y_tr_pred)],
                    "set": ["test", "train"]}
        df = pd.DataFrame(resultados)
        df["modelo"] = tipo_modelo
        return df

    def grid_s_gradient(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= GradientBoostingRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

    def grid_s_forest(self, dict, model):
        self.dict = dict
        self.model = model
        
        X = self.dataf.drop(self.col_pred, axis = 1)
        y = self.dataf[self.col_pred]
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
        
        gs = GridSearchCV(
            estimator= RandomForestRegressor(), 
            param_grid= self.dict, 
            cv=10, 
            verbose=-1,
            return_train_score = True,
            scoring="neg_mean_squared_error")
        gs.fit(x_train, y_train)
        mejor_modelo = gs.best_estimator_
        y_pred_test_dt2 = mejor_modelo.predict(x_test)
        y_pred_train_dt2 = mejor_modelo.predict(x_train)
        dt_results = self.metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, self.model)
        return mejor_modelo, dt_results

In [5]:
# Llamamos a la clase
rg_est = Regres_lineal(df_est, "registered")

In [6]:
features_est, depth_est = rg_est.tree_param()

In [7]:
features_est #El número máx será 3

3.1622776601683795

In [8]:
depth_est # La profundidad máxima será 20

18

### Empezamos las diferentes pruebas

In [9]:
param1 = {"max_depth": [16, 18, 20], 
        "max_features": [3,4],
        "min_samples_split": [50, 100],
        "min_samples_leaf": [50,100]} 

In [10]:
best_model1, result1 = rg_est.grid_s_gradient(param1, "GradientBoosting Estand I")

In [11]:
best_model1

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=50,
                          min_samples_split=100)

In [12]:
result1

,MAE,MSE,RMSE,R2,set,modelo
0,576.315580,541803.648122,736.073127,0.772417,test,GradientBoosting Estand I
1,463.851411,360171.293983,600.142728,0.851765,train,GradientBoosting Estand I


In [13]:
param2 = {"max_depth": [4,6,8,10], 
        "max_features": [2, 3,4],
        "min_samples_split": [ 40, 50, 60],
        "min_samples_leaf": [10, 20]} 

In [14]:
best_model2, result2 = rg_est.grid_s_gradient(param2, "GradientBoosting Estand II")

In [15]:
best_model2

GradientBoostingRegressor(max_depth=4, max_features=4, min_samples_leaf=10,
                          min_samples_split=60)

In [16]:
result2

,MAE,MSE,RMSE,R2,set,modelo
0,544.520079,490032.237429,700.023026,0.794163,test,GradientBoosting Estand II
1,420.182357,290938.840594,539.387468,0.880259,train,GradientBoosting Estand II


In [17]:
param3 = {"max_depth": [ 12, 14, 16, 18], 
        "max_features": [3,4],
        "min_samples_split": [10, 20, 40],
        "min_samples_leaf": [10, 20, 40]} 

In [18]:
best_model3, result3 = rg_est.grid_s_forest(param3, "RandForest Estand I")

In [19]:
best_model3

RandomForestRegressor(max_depth=12, max_features=4, min_samples_leaf=10,
                      min_samples_split=20)

In [20]:
result3

,MAE,MSE,RMSE,R2,set,modelo
0,623.782082,593310.210483,770.266324,0.750782,test,RandForest Estand I
1,573.847672,521329.872651,722.031767,0.785437,train,RandForest Estand I


In [21]:
param4 = {"max_depth": [12, 14], 
        "max_features": [4,5],
        "min_samples_split": [ 40, 50],
        "min_samples_leaf": [10, 20,]} 

In [22]:
best_model4, result4 = rg_est.grid_s_forest(param4, "RandForest Estand II")

In [23]:
best_model4

RandomForestRegressor(max_depth=12, max_features=5, min_samples_leaf=10,
                      min_samples_split=40)

In [24]:
result4

,MAE,MSE,RMSE,R2,set,modelo
0,630.660386,616499.916979,785.175087,0.741041,test,RandForest Estand II
1,599.444265,573872.896324,757.543990,0.763812,train,RandForest Estand II


### Cargamos el archivo sin estandarizar

In [25]:
df_sinest = pd.read_csv("../datos/07-bikes_enc_sinest_sinoutliers.csv", index_col = 0)
df_sinest.head()

,season,yr,mnth,holiday,weekday,workingday,weathersit,atemp,hum,windspeed,registered
0,0,1,0,0,0,1,2,18.18125,80.5833,10.749882,654
1,0,1,0,7,2,2,2,17.68695,69.6087,16.652113,670
2,0,1,1,7,2,2,4,9.47025,43.7273,16.636703,1229
3,1,1,1,7,1,1,4,10.60610,59.0435,10.739832,1454
4,1,1,2,7,1,2,4,11.46350,43.6957,12.522300,1518


In [26]:
rg_sinest = Regres_lineal(df_sinest, "registered")

In [27]:
features_sinest, depth_sinest = rg_sinest.tree_param()

In [28]:
features_sinest #El número máx será 3

3.1622776601683795

In [29]:
depth_sinest # La profundidad máxima será 20

18

In [30]:
best_model5, result5 = rg_sinest.grid_s_gradient(param1, "GradientBoosting SinEstand I")

In [31]:
result5

,MAE,MSE,RMSE,R2,set,modelo
0,578.491352,530248.672886,728.181758,0.777270,test,GradientBoosting SinEstand I
1,473.089032,372764.509287,610.544437,0.846582,train,GradientBoosting SinEstand I


In [32]:
best_model6, result6 = rg_sinest.grid_s_gradient(param2, "GradientBoosting SinEstand II")

In [33]:
result6

,MAE,MSE,RMSE,R2,set,modelo
0,556.863221,510411.254857,714.430721,0.785603,test,GradientBoosting SinEstand II
1,427.504515,306376.468234,553.512844,0.873905,train,GradientBoosting SinEstand II


In [34]:
best_model7, result7 = rg_sinest.grid_s_forest(param3, "RandForest SinEstand I")

In [35]:
result7

,MAE,MSE,RMSE,R2,set,modelo
0,625.019251,593637.560667,770.478787,0.750644,test,RandForest SinEstand I
1,570.305866,515569.392350,718.031610,0.787808,train,RandForest SinEstand I


In [36]:
best_model8, result8 = rg_sinest.grid_s_forest(param4, "RandForest SinEstand II")

In [37]:
result8

,MAE,MSE,RMSE,R2,set,modelo
0,623.716333,614321.078734,783.786373,0.741956,test,RandForest SinEstand II
1,596.843743,567866.070861,753.568889,0.766285,train,RandForest SinEstand II


In [38]:
param5 = {"max_depth": [22], 
        "max_features": [4],
        "min_samples_split": [8],
        "min_samples_leaf": [8]}

In [39]:
best_model9, result9 = rg_sinest.grid_s_forest(param5, "RandForest SinEstand III")

In [40]:
result9

,MAE,MSE,RMSE,R2,set,modelo
0,607.385560,573767.340899,757.474317,0.758991,test,RandForest SinEstand III
1,538.080793,466187.296190,682.779098,0.808132,train,RandForest SinEstand III


In [41]:
best_model10, result10 = rg_est.grid_s_forest(param5, "RandForest Estand III")

In [42]:
result10

,MAE,MSE,RMSE,R2,set,modelo
0,605.762694,573463.341110,757.273624,0.759118,test,RandForest Estand III
1,541.281672,475101.550268,689.276106,0.804463,train,RandForest Estand III


In [43]:
param6 = {"max_depth": [18,20], 
        "max_features": [3,4],
        "min_samples_split": [8, 15],
        "min_samples_leaf": [8, 15]}

In [44]:
best_model11, result11 = rg_est.grid_s_forest(param6, "RandForest Estand IV")

In [45]:
best_model11

RandomForestRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                      min_samples_split=8)

In [46]:
result11

,MAE,MSE,RMSE,R2,set,modelo
0,609.229007,579095.483751,760.983235,0.756753,test,RandForest Estand IV
1,541.316712,469394.296064,685.123563,0.806812,train,RandForest Estand IV


In [47]:
best_model12, result12 = rg_est.grid_s_gradient(param5, "GradientBoosting Estand III")

In [48]:
result12

,MAE,MSE,RMSE,R2,set,modelo
0,559.501698,497909.542274,705.627056,0.790854,test,GradientBoosting Estand III
1,104.866855,22311.988717,149.371981,0.990817,train,GradientBoosting Estand III


In [49]:
best_model13, result13 = rg_sinest.grid_s_gradient(param5, "GradientBoosting SinEstand III")

In [50]:
best_model13

GradientBoostingRegressor(max_depth=22, max_features=4, min_samples_leaf=8,
                          min_samples_split=8)

In [51]:
result13

,MAE,MSE,RMSE,R2,set,modelo
0,576.254746,539874.779924,734.761716,0.773227,test,GradientBoosting SinEstand III
1,103.522394,21594.053067,146.949151,0.991113,train,GradientBoosting SinEstand III


In [52]:
param7 = {"max_depth": [18,20], 
        "max_features": [4],
        "min_samples_split": [8, 16],
        "min_samples_leaf": [8]}

In [53]:
best_model14, result14 = rg_est.grid_s_gradient(param7, "GradientBoosting Estand IV")

In [54]:
best_model14

GradientBoostingRegressor(max_depth=20, max_features=4, min_samples_leaf=8,
                          min_samples_split=8)

In [55]:
result14

,MAE,MSE,RMSE,R2,set,modelo
0,561.421735,519523.306041,720.779652,0.781776,test,GradientBoosting Estand IV
1,109.446409,24612.758303,156.884538,0.989870,train,GradientBoosting Estand IV


In [56]:
param17 = {"max_depth": [4], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model15, result15 = rg_est.grid_s_gradient(param17, "GradientBoosting Estand IV")

result15

,MAE,MSE,RMSE,R2,set,modelo
0,532.735060,491484.559376,701.059598,0.793553,test,GradientBoosting Estand IV
1,416.048508,288327.850007,536.961684,0.881334,train,GradientBoosting Estand IV


In [57]:
param17 = {"max_depth": [4], 
        "max_features": [3],
        "min_samples_split": [50],
        "min_samples_leaf": [10]}

best_model16, result16 = rg_sinest.grid_s_gradient(param17, "GradientBoosting SinEstand IV")

result16

,MAE,MSE,RMSE,R2,set,modelo
0,554.599625,525567.958164,724.960660,0.779237,test,GradientBoosting SinEstand IV
1,413.104078,287993.682218,536.650428,0.881471,train,GradientBoosting SinEstand IV


In [58]:
with open ("../datos/model_prediccion.pkl", "wb") as fp:
    pickle.dump(best_model16, fp)